In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://occupationalhealthblog.net/dentist/Seoul/Gangnam-gu/'

all_data = []

# 강남구 페이지 번호
for i in range(2113, 2127):
    url = f'{base_url}{i}/'

    response = requests.get(url)

    # 병원명, 의사수, 전화번호, 주소
    hospital_names = []
    doctor_counts = []
    phone_numbers = []
    addresses = []
    links = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # 병원명 및 링크 크롤링
        a_tags = soup.find_all('a', class_='placeName text-info card-text h3')
        for a_tag in a_tags:
            hospital_names.append(a_tag.find('h3', class_='py-1 my-0').text)
            links.append(a_tag['href'])

        # 의사수, 전화번호, 주소 크롤링
        ul_tags = soup.find_all('ul', class_='px-3')
        for ul in ul_tags:
            li_tags = ul.find_all('li')

            # 각 데이터 추출
            doctor_count = li_tags[0].text.replace('의사수: ', '').strip() if len(li_tags) > 0 else ''
            phone_number = li_tags[1].find('a').text.strip() if len(li_tags) > 1 and li_tags[1].find('a') else ''
            address = li_tags[2].text.strip() if len(li_tags) > 2 else ''

            doctor_counts.append(doctor_count)
            phone_numbers.append(phone_number)
            addresses.append(address)

        lunch_times = []
        reception_weekday_times = []
        reception_weekend_times = []
        monday_hours = []
        tuesday_hours = []
        wednesday_hours = []
        thursday_hours = []
        friday_hours = []
        saturday_hours = []
        sunday_hours = []

        for link in links:
            sub_response = requests.get(link)
            if sub_response.status_code == 200:
                sub_soup = BeautifulSoup(sub_response.text, 'html.parser')
                ul_sub_tags = sub_soup.find_all('ul', class_='list-group list-group-flush')

                lunch_time = ""
                reception_weekday_time = ""
                reception_weekend_time = ""
                monday_hour = ""
                tuesday_hour = ""
                wednesday_hour = ""
                thursday_hour = ""
                friday_hour = ""
                saturday_hour = ""
                sunday_hour = ""

                for ul_sub_tag in ul_sub_tags:
                    li_tags = ul_sub_tag.find_all('li', class_='list-group-item')
                    for li in li_tags:
                        if '점심시간' in li.text:
                            sub_ul = li.find_next('ul')
                            if sub_ul:
                                sub_li_tags = sub_ul.find_all('li')
                                for sub_li in sub_li_tags:
                                    if '점심시간(평일)' in sub_li.text:
                                        lunch_time = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""
                        elif '접수시간' in li.text:
                            sub_ul = li.find_next('ul')
                            if sub_ul:
                                sub_li_tags = sub_ul.find_all('li')
                                for sub_li in sub_li_tags:
                                    if '접수시간(평일)' in sub_li.text:
                                        reception_weekday_time = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""
                                    elif '접수시간(토요일)' in sub_li.text:
                                        reception_weekend_time = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""
                        elif '진료시간' in li.text:
                            sub_ul = li.find_next('ul')
                            if sub_ul:
                                sub_li_tags = sub_ul.find_all('li')
                                for sub_li in sub_li_tags:
                                    if '월' in sub_li.text:
                                        monday_hour = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""
                                    elif '화' in sub_li.text:
                                        tuesday_hour = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""
                                    elif '수' in sub_li.text:
                                        wednesday_hour = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""
                                    elif '목' in sub_li.text:
                                        thursday_hour = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""
                                    elif '금' in sub_li.text:
                                        friday_hour = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""
                                    elif '토' in sub_li.text:
                                        saturday_hour = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""
                                    elif '일' in sub_li.text:
                                        sunday_hour = sub_li.text.split(' ')[-1] if ' ' in sub_li.text else ""

                lunch_times.append(lunch_time)
                reception_weekday_times.append(reception_weekday_time)
                reception_weekend_times.append(reception_weekend_time)
                monday_hours.append(monday_hour)
                tuesday_hours.append(tuesday_hour)
                wednesday_hours.append(wednesday_hour)
                thursday_hours.append(thursday_hour)
                friday_hours.append(friday_hour)
                saturday_hours.append(saturday_hour)
                sunday_hours.append(sunday_hour)
            else:
                lunch_times.append("")
                reception_weekday_times.append("")
                reception_weekend_times.append("")
                monday_hours.append("")
                tuesday_hours.append("")
                wednesday_hours.append("")
                thursday_hours.append("")
                friday_hours.append("")
                saturday_hours.append("")
                sunday_hours.append("")

        # 길이가 일치하지 않으면 가장 긴 길이에 맞추기
        max_length = max(len(hospital_names), len(doctor_counts), len(phone_numbers), len(addresses), len(lunch_times), len(reception_weekday_times), len(reception_weekend_times), len(monday_hours), len(tuesday_hours), len(wednesday_hours), len(thursday_hours), len(friday_hours), len(saturday_hours), len(sunday_hours))

        hospital_names.extend([None] * (max_length - len(hospital_names)))
        doctor_counts.extend([''] * (max_length - len(doctor_counts)))
        phone_numbers.extend([''] * (max_length - len(phone_numbers)))
        addresses.extend([''] * (max_length - len(addresses)))
        lunch_times.extend([''] * (max_length - len(lunch_times)))
        reception_weekday_times.extend([''] * (max_length - len(reception_weekday_times)))
        reception_weekend_times.extend([''] * (max_length - len(reception_weekend_times)))
        monday_hours.extend([''] * (max_length - len(monday_hours)))
        tuesday_hours.extend([''] * (max_length - len(tuesday_hours)))
        wednesday_hours.extend([''] * (max_length - len(wednesday_hours)))
        thursday_hours.extend([''] * (max_length - len(thursday_hours)))
        friday_hours.extend([''] * (max_length - len(friday_hours)))
        saturday_hours.extend([''] * (max_length - len(saturday_hours)))
        sunday_hours.extend([''] * (max_length - len(sunday_hours)))

        # 데이터프레임 생성
        df = pd.DataFrame({
            '병원명': hospital_names,
            '의사수': doctor_counts,
            '전화번호': phone_numbers,
            '주소': addresses,
            '점심시간(평일)': lunch_times,
            '접수시간(평일)': reception_weekday_times,
            '접수시간(토)': reception_weekend_times,
            '진료시간(월)': monday_hours,
            '진료시간(화)': tuesday_hours,
            '진료시간(수)': wednesday_hours,
            '진료시간(목)': thursday_hours,
            '진료시간(금)': friday_hours,
            '진료시간(토)': saturday_hours,
            '진료시간(일)': sunday_hours
        })

        # 병원명이 None인 행 제거
        df.dropna(subset=['병원명'], inplace=True)

        # 크롤링한 데이터 추가
        all_data.append(df)

    else:
        print(f"웹페이지 요청 실패: {response.status_code} at {url}")

# 모든 데이터 병합
merged_data = pd.concat(all_data, ignore_index=True)

merged_data = merged_data[['병원명', '의사수', '전화번호', '주소', '점심시간(평일)', '접수시간(평일)', '접수시간(토)', '진료시간(월)', '진료시간(화)', '진료시간(수)', '진료시간(목)', '진료시간(금)', '진료시간(토)', '진료시간(일)']]

merged_data


,병원명,의사수,전화번호,주소,점심시간(평일),접수시간(평일),접수시간(토),진료시간(월),진료시간(화),진료시간(수),진료시간(목),진료시간(금),진료시간(토),진료시간(일)
0,강남연세치과의원,2,02-529-1117,"서울특별시 강남구 선릉로 30, 개포빌딩 2층 2호 (개포동)",12:00~2:00,,,18:00,18:00,18:00,18:00,18:00,12:30,
1,강남편안한치과의원,1,02-573-2879,"서울특별시 강남구 언주로7길 6, 2층 206,207호 (개포동, 개포자이르네)",,,,,,,,,,
2,개포어울림치과의원,1,02-3412-2876,"서울특별시 강남구 개포로 504, 메트하임 4층 403, 404호 (개포동)",13시00분~14시00분,09시30분~18시30분,,18:30,18:30,18:30,18:30,18:30,18:30,
3,개포인치과의원,1,3411-2804,"서울특별시 강남구 삼성로 40, 202,203호 (개포동)",13시00분~14시00분,10시00분~19시00분,휴진),19:00,19:00,19:00,19:00,19:00,13:00,
4,리솜치과의원,1,02-2058-2828,"서울특별시 강남구 개포로 265, 개포동 메디시스빌딩 403호 (개포동)",,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,청담유펜치과의원,1,02-548-7316,"서울특별시 강남구 영동대로138길 15, 103호 (청담동)",,,,,,,,,,
540,청담이사랑치과의원,1,02-517-8275,"서울특별시 강남구 삼성로 766, 2층 (청담동)",13시00분~14시00분,09시30분~18시00분,09시30분~14시00분,18:30,18:30,18:30,18:30,18:30,14:30,
541,청담치과의원,1,514-6480,"서울특별시 강남구 도산대로 513, 양원빌딩 4층 (청담동)",30분,,,18:00,18:00,18:00,18:00,18:00,15:00,
542,한미치과의원,2,549-9841,"서울특별시 강남구 도산대로55길 29, 4층 (청담동)",13시00분~14시00분,09시30분~18시30분,09시30분~16시00분,18:30,18:30,18:30,18:30,18:30,16:00,


In [ ]:
# 전화번호 컬럼에서 '02-'가 없는 경우 앞에 추가
merged_data['전화번호'] = merged_data['전화번호'].apply(lambda x: x if pd.isna(x) or x.startswith('02-') else '02-' + x)

# 주소 컬럼에서 첫 번째 쉼표 이후 문자열 삭제해서 주소_삭제라는 컬럼 생성(삭제할예)
merged_data['주소_삭제'] = merged_data['주소'].apply(lambda x: x.split(',')[0] if pd.notna(x) else x)

merged_data.reset_index(drop=True,inplace=True)

In [ ]:
# 위도, 경도 컬럼
merged_data['latitude'] = 0.0
merged_data['longitude'] = 0.0

In [ ]:
merged_data

,병원명,의사수,전화번호,주소,점심시간(평일),접수시간(평일),접수시간(토),진료시간(월),진료시간(화),진료시간(수),진료시간(목),진료시간(금),진료시간(토),진료시간(일),주소_삭제,latitude,longitude
0,강남연세치과의원,2,02-529-1117,"서울특별시 강남구 선릉로 30, 개포빌딩 2층 2호 (개포동)",12:00~2:00,,,18:00,18:00,18:00,18:00,18:00,12:30,,서울특별시 강남구 선릉로 30,0.0,0.0
1,강남편안한치과의원,1,02-573-2879,"서울특별시 강남구 언주로7길 6, 2층 206,207호 (개포동, 개포자이르네)",,,,,,,,,,,서울특별시 강남구 언주로7길 6,0.0,0.0
2,개포어울림치과의원,1,02-3412-2876,"서울특별시 강남구 개포로 504, 메트하임 4층 403, 404호 (개포동)",13시00분~14시00분,09시30분~18시30분,,18:30,18:30,18:30,18:30,18:30,18:30,,서울특별시 강남구 개포로 504,0.0,0.0
3,개포인치과의원,1,02-3411-2804,"서울특별시 강남구 삼성로 40, 202,203호 (개포동)",13시00분~14시00분,10시00분~19시00분,휴진),19:00,19:00,19:00,19:00,19:00,13:00,,서울특별시 강남구 삼성로 40,0.0,0.0
4,리솜치과의원,1,02-2058-2828,"서울특별시 강남구 개포로 265, 개포동 메디시스빌딩 403호 (개포동)",,,,,,,,,,,서울특별시 강남구 개포로 265,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,청담유펜치과의원,1,02-548-7316,"서울특별시 강남구 영동대로138길 15, 103호 (청담동)",,,,,,,,,,,서울특별시 강남구 영동대로138길 15,0.0,0.0
540,청담이사랑치과의원,1,02-517-8275,"서울특별시 강남구 삼성로 766, 2층 (청담동)",13시00분~14시00분,09시30분~18시00분,09시30분~14시00분,18:30,18:30,18:30,18:30,18:30,14:30,,서울특별시 강남구 삼성로 766,0.0,0.0
541,청담치과의원,1,02-514-6480,"서울특별시 강남구 도산대로 513, 양원빌딩 4층 (청담동)",30분,,,18:00,18:00,18:00,18:00,18:00,15:00,,서울특별시 강남구 도산대로 513,0.0,0.0
542,한미치과의원,2,02-549-9841,"서울특별시 강남구 도산대로55길 29, 4층 (청담동)",13시00분~14시00분,09시30분~18시30분,09시30분~16시00분,18:30,18:30,18:30,18:30,18:30,16:00,,서울특별시 강남구 도산대로55길 29,0.0,0.0


In [ ]:
from geopy.geocoders import Nominatim

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo_local = Nominatim(user_agent='South Korea')  #지역설정
        location = geo_local.geocode(address)
        geo = [location.latitude, location.longitude]
        return geo

    except:
        return [0,0]

# 실행
for idx,addr in enumerate(merged_data["주소_삭제"]):
    merged_data.loc[idx,'latitude'] = geocoding(addr)[0]
    merged_data.loc[idx,'longitude'] = geocoding(addr)[1]

In [ ]:
merged_data

,병원명,의사수,전화번호,주소,점심시간(평일),접수시간(평일),접수시간(토),진료시간(월),진료시간(화),진료시간(수),진료시간(목),진료시간(금),진료시간(토),진료시간(일),주소_삭제,latitude,longitude
0,강남연세치과의원,2,02-529-1117,"서울특별시 강남구 선릉로 30, 개포빌딩 2층 2호 (개포동)",12:00~2:00,,,18:00,18:00,18:00,18:00,18:00,12:30,,서울특별시 강남구 선릉로 30,37.482415,127.061184
1,강남편안한치과의원,1,02-573-2879,"서울특별시 강남구 언주로7길 6, 2층 206,207호 (개포동, 개포자이르네)",,,,,,,,,,,서울특별시 강남구 언주로7길 6,37.476938,127.055147
2,개포어울림치과의원,1,02-3412-2876,"서울특별시 강남구 개포로 504, 메트하임 4층 403, 404호 (개포동)",13시00분~14시00분,09시30분~18시30분,,18:30,18:30,18:30,18:30,18:30,18:30,,서울특별시 강남구 개포로 504,37.489101,127.067119
3,개포인치과의원,1,02-3411-2804,"서울특별시 강남구 삼성로 40, 202,203호 (개포동)",13시00분~14시00분,10시00분~19시00분,휴진),19:00,19:00,19:00,19:00,19:00,13:00,,서울특별시 강남구 삼성로 40,37.510063,127.054864
4,리솜치과의원,1,02-2058-2828,"서울특별시 강남구 개포로 265, 개포동 메디시스빌딩 403호 (개포동)",,,,,,,,,,,서울특별시 강남구 개포로 265,37.480894,127.053009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,청담유펜치과의원,1,02-548-7316,"서울특별시 강남구 영동대로138길 15, 103호 (청담동)",,,,,,,,,,,서울특별시 강남구 영동대로138길 15,37.523178,127.056442
540,청담이사랑치과의원,1,02-517-8275,"서울특별시 강남구 삼성로 766, 2층 (청담동)",13시00분~14시00분,09시30분~18시00분,09시30분~14시00분,18:30,18:30,18:30,18:30,18:30,14:30,,서울특별시 강남구 삼성로 766,37.523209,127.048254
541,청담치과의원,1,02-514-6480,"서울특별시 강남구 도산대로 513, 양원빌딩 4층 (청담동)",30분,,,18:00,18:00,18:00,18:00,18:00,15:00,,서울특별시 강남구 도산대로 513,37.516237,127.019637
542,한미치과의원,2,02-549-9841,"서울특별시 강남구 도산대로55길 29, 4층 (청담동)",13시00분~14시00분,09시30분~18시30분,09시30분~16시00분,18:30,18:30,18:30,18:30,18:30,16:00,,서울특별시 강남구 도산대로55길 29,37.525447,127.040578


In [ ]:
import re

def extract_dong(address):

    matches = re.findall(r'\(([^)]+)\)', address)
    for match in matches:
        if '동' in match:
            # '동'까지 포함하여 반환
            return match.split('동')[0] + '동'
    return None

# '동' 컬럼 생성
merged_data['동'] = merged_data['주소'].apply(extract_dong)

In [ ]:
merged_data

,병원명,의사수,전화번호,주소,점심시간(평일),접수시간(평일),접수시간(토),진료시간(월),진료시간(화),진료시간(수),진료시간(목),진료시간(금),진료시간(토),진료시간(일),주소_삭제,latitude,longitude,동
0,강남연세치과의원,2,02-529-1117,"서울특별시 강남구 선릉로 30, 개포빌딩 2층 2호 (개포동)",12:00~2:00,,,18:00,18:00,18:00,18:00,18:00,12:30,,서울특별시 강남구 선릉로 30,37.482415,127.061184,개포동
1,강남편안한치과의원,1,02-573-2879,"서울특별시 강남구 언주로7길 6, 2층 206,207호 (개포동, 개포자이르네)",,,,,,,,,,,서울특별시 강남구 언주로7길 6,37.476938,127.055147,개포동
2,개포어울림치과의원,1,02-3412-2876,"서울특별시 강남구 개포로 504, 메트하임 4층 403, 404호 (개포동)",13시00분~14시00분,09시30분~18시30분,,18:30,18:30,18:30,18:30,18:30,18:30,,서울특별시 강남구 개포로 504,37.489101,127.067119,개포동
3,개포인치과의원,1,02-3411-2804,"서울특별시 강남구 삼성로 40, 202,203호 (개포동)",13시00분~14시00분,10시00분~19시00분,휴진),19:00,19:00,19:00,19:00,19:00,13:00,,서울특별시 강남구 삼성로 40,37.510063,127.054864,개포동
4,리솜치과의원,1,02-2058-2828,"서울특별시 강남구 개포로 265, 개포동 메디시스빌딩 403호 (개포동)",,,,,,,,,,,서울특별시 강남구 개포로 265,37.480894,127.053009,개포동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,청담유펜치과의원,1,02-548-7316,"서울특별시 강남구 영동대로138길 15, 103호 (청담동)",,,,,,,,,,,서울특별시 강남구 영동대로138길 15,37.523178,127.056442,청담동
540,청담이사랑치과의원,1,02-517-8275,"서울특별시 강남구 삼성로 766, 2층 (청담동)",13시00분~14시00분,09시30분~18시00분,09시30분~14시00분,18:30,18:30,18:30,18:30,18:30,14:30,,서울특별시 강남구 삼성로 766,37.523209,127.048254,청담동
541,청담치과의원,1,02-514-6480,"서울특별시 강남구 도산대로 513, 양원빌딩 4층 (청담동)",30분,,,18:00,18:00,18:00,18:00,18:00,15:00,,서울특별시 강남구 도산대로 513,37.516237,127.019637,청담동
542,한미치과의원,2,02-549-9841,"서울특별시 강남구 도산대로55길 29, 4층 (청담동)",13시00분~14시00분,09시30분~18시30분,09시30분~16시00분,18:30,18:30,18:30,18:30,18:30,16:00,,서울특별시 강남구 도산대로55길 29,37.525447,127.040578,청담동


In [ ]:
merged_data['동'].value_counts()

동
역삼동     125
신사동     102
대치동      75
논현동      65
삼성동      45
도곡동      33
청담동      32
개포동      30
일원동      17
수서동       8
세곡동       5
압구정동      3
율현동       2
자곡동       2
Name: count, dtype: int64

In [ ]:
merged_data

,병원명,의사수,전화번호,주소,점심시간(평일),접수시간(평일),접수시간(토),진료시간(월),진료시간(화),진료시간(수),진료시간(목),진료시간(금),진료시간(토),진료시간(일),주소_삭제,latitude,longitude,동
0,강남연세치과의원,2,02-529-1117,"서울특별시 강남구 선릉로 30, 개포빌딩 2층 2호 (개포동)",12:00~2:00,,,18:00,18:00,18:00,18:00,18:00,12:30,,서울특별시 강남구 선릉로 30,37.482415,127.061184,개포동
1,강남편안한치과의원,1,02-573-2879,"서울특별시 강남구 언주로7길 6, 2층 206,207호 (개포동, 개포자이르네)",,,,,,,,,,,서울특별시 강남구 언주로7길 6,37.476938,127.055147,개포동
2,개포어울림치과의원,1,02-3412-2876,"서울특별시 강남구 개포로 504, 메트하임 4층 403, 404호 (개포동)",13시00분~14시00분,09시30분~18시30분,,18:30,18:30,18:30,18:30,18:30,18:30,,서울특별시 강남구 개포로 504,37.489101,127.067119,개포동
3,개포인치과의원,1,02-3411-2804,"서울특별시 강남구 삼성로 40, 202,203호 (개포동)",13시00분~14시00분,10시00분~19시00분,휴진),19:00,19:00,19:00,19:00,19:00,13:00,,서울특별시 강남구 삼성로 40,37.510063,127.054864,개포동
4,리솜치과의원,1,02-2058-2828,"서울특별시 강남구 개포로 265, 개포동 메디시스빌딩 403호 (개포동)",,,,,,,,,,,서울특별시 강남구 개포로 265,37.480894,127.053009,개포동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,청담유펜치과의원,1,02-548-7316,"서울특별시 강남구 영동대로138길 15, 103호 (청담동)",,,,,,,,,,,서울특별시 강남구 영동대로138길 15,37.523178,127.056442,청담동
540,청담이사랑치과의원,1,02-517-8275,"서울특별시 강남구 삼성로 766, 2층 (청담동)",13시00분~14시00분,09시30분~18시00분,09시30분~14시00분,18:30,18:30,18:30,18:30,18:30,14:30,,서울특별시 강남구 삼성로 766,37.523209,127.048254,청담동
541,청담치과의원,1,02-514-6480,"서울특별시 강남구 도산대로 513, 양원빌딩 4층 (청담동)",30분,,,18:00,18:00,18:00,18:00,18:00,15:00,,서울특별시 강남구 도산대로 513,37.516237,127.019637,청담동
542,한미치과의원,2,02-549-9841,"서울특별시 강남구 도산대로55길 29, 4층 (청담동)",13시00분~14시00분,09시30분~18시30분,09시30분~16시00분,18:30,18:30,18:30,18:30,18:30,16:00,,서울특별시 강남구 도산대로55길 29,37.525447,127.040578,청담동


In [ ]:
merged_data.drop("주소_삭제",axis=1,inplace=True)
merged_data['주소'] = merged_data['주소'].str.extract(r'^(.*?)(?: \()')

In [ ]:
merged_data

,병원명,의사수,전화번호,주소,점심시간(평일),접수시간(평일),접수시간(토),진료시간(월),진료시간(화),진료시간(수),진료시간(목),진료시간(금),진료시간(토),진료시간(일),latitude,longitude,동
0,강남연세치과의원,2,02-529-1117,"서울특별시 강남구 선릉로 30, 개포빌딩 2층 2호",12:00~2:00,,,18:00,18:00,18:00,18:00,18:00,12:30,,37.482415,127.061184,개포동
1,강남편안한치과의원,1,02-573-2879,"서울특별시 강남구 언주로7길 6, 2층 206,207호",,,,,,,,,,,37.476938,127.055147,개포동
2,개포어울림치과의원,1,02-3412-2876,"서울특별시 강남구 개포로 504, 메트하임 4층 403, 404호",13시00분~14시00분,09시30분~18시30분,,18:30,18:30,18:30,18:30,18:30,18:30,,37.489101,127.067119,개포동
3,개포인치과의원,1,02-3411-2804,"서울특별시 강남구 삼성로 40, 202,203호",13시00분~14시00분,10시00분~19시00분,휴진),19:00,19:00,19:00,19:00,19:00,13:00,,37.510063,127.054864,개포동
4,리솜치과의원,1,02-2058-2828,"서울특별시 강남구 개포로 265, 개포동 메디시스빌딩 403호",,,,,,,,,,,37.480894,127.053009,개포동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,청담유펜치과의원,1,02-548-7316,"서울특별시 강남구 영동대로138길 15, 103호",,,,,,,,,,,37.523178,127.056442,청담동
540,청담이사랑치과의원,1,02-517-8275,"서울특별시 강남구 삼성로 766, 2층",13시00분~14시00분,09시30분~18시00분,09시30분~14시00분,18:30,18:30,18:30,18:30,18:30,14:30,,37.523209,127.048254,청담동
541,청담치과의원,1,02-514-6480,"서울특별시 강남구 도산대로 513, 양원빌딩 4층",30분,,,18:00,18:00,18:00,18:00,18:00,15:00,,37.516237,127.019637,청담동
542,한미치과의원,2,02-549-9841,"서울특별시 강남구 도산대로55길 29, 4층",13시00분~14시00분,09시30분~18시30분,09시30분~16시00분,18:30,18:30,18:30,18:30,18:30,16:00,,37.525447,127.040578,청담동


In [ ]:
# output_file = '강남구_치과.xlsx'
# merged_data.to_excel(output_file, index=False, engine='openpyxl')

In [ ]:
# pd.read_excel("/content/강남구_치과.xlsx")

,병원명,의사수,전화번호,주소,점심시간(평일),접수시간(평일),접수시간(토),진료시간(월),진료시간(화),진료시간(수),진료시간(목),진료시간(금),진료시간(토),진료시간(일),latitude,longitude,동
0,강남연세치과의원,2,02-529-1117,"서울특별시 강남구 선릉로 30, 개포빌딩 2층 2호",12:00~2:00,NaN,NaN,18:00,18:00,18:00,18:00,18:00,12:30,NaN,37.482415,127.061184,개포동
1,강남편안한치과의원,1,02-573-2879,"서울특별시 강남구 언주로7길 6, 2층 206,207호",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.476938,127.055147,개포동
2,개포어울림치과의원,1,02-3412-2876,"서울특별시 강남구 개포로 504, 메트하임 4층 403, 404호",13시00분~14시00분,09시30분~18시30분,NaN,18:30,18:30,18:30,18:30,18:30,18:30,NaN,37.489101,127.067119,개포동
3,개포인치과의원,1,02-3411-2804,"서울특별시 강남구 삼성로 40, 202,203호",13시00분~14시00분,10시00분~19시00분,휴진),19:00,19:00,19:00,19:00,19:00,13:00,NaN,37.510063,127.054864,개포동
4,리솜치과의원,1,02-2058-2828,"서울특별시 강남구 개포로 265, 개포동 메디시스빌딩 403호",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.480894,127.053009,개포동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,청담유펜치과의원,1,02-548-7316,"서울특별시 강남구 영동대로138길 15, 103호",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.523178,127.056442,청담동
540,청담이사랑치과의원,1,02-517-8275,"서울특별시 강남구 삼성로 766, 2층",13시00분~14시00분,09시30분~18시00분,09시30분~14시00분,18:30,18:30,18:30,18:30,18:30,14:30,NaN,37.523209,127.048254,청담동
541,청담치과의원,1,02-514-6480,"서울특별시 강남구 도산대로 513, 양원빌딩 4층",30분,NaN,NaN,18:00,18:00,18:00,18:00,18:00,15:00,NaN,37.516237,127.019637,청담동
542,한미치과의원,2,02-549-9841,"서울특별시 강남구 도산대로55길 29, 4층",13시00분~14시00분,09시30분~18시30분,09시30분~16시00분,18:30,18:30,18:30,18:30,18:30,16:00,NaN,37.525447,127.040578,청담동
